# PROJECT-2. Анализ вакансий из HeadHunter

### 3. Предварительный анализ данных

In [2]:
import pandas as pd
import psycopg2

In [3]:
# хороший тон при фиксации изменений в github не публиковать код с открытыми данными для входа
# для запуска кода поместите данные, которые указаны на странице курса в эту ячейку
# при размещении в github удалите эту ячейку

DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT = 0

In [4]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

**Задание 3.1.** Сколько вакансий есть в базе?

In [65]:
query = f'''select count(*) as vacancies_count
            from vacancies 
         '''
# лучше использовать f-строки - так удобнее передавать параметры
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\3768891433.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count
0,49197


В базе HeadHunter 49197 вакансий.

**Задание 3.2.** Сколько работодателей?

In [104]:
query = '''
    select count(*) as employers_count 
    from employers
    '''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1064020809.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employers_count
0,23501


В базе HeadHunter представлен 23501 работодатель.

**Задание 3.3.** Сколько регионов?

In [73]:
query = 'select count(*) as areas_count from areas'
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2675087203.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,areas_count
0,1362


В базе HeadHunter представлено 1362 региона.

**Задание 3.4.** Сколько сфер деятельности?

In [75]:
query = 'select count(*) as industries_count from industries'
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\987574708.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,industries_count
0,294


В базе HeadHunter представлено 294 сферы деятельности.

*База HeadHunter - крупная платформа по поиску работы. В ней представлены вакансии по 1362 регионам от 23501 работодателя. Также большой охват по сферам деятельности: 294.*

## 4. Детальный анализ вакансий

**Задание 4.1**

Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Выберите пятёрку лидеров по количеству вакансий:

In [77]:
query = '''
    select a.name as area, count(v.id) as cnt
    from vacancies v 
        join areas a on v.area_id = a.id
    group by a.name
    order by cnt desc, area
    limit 5
    '''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\968293266.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892



Пятерка лидеров по количеству вакансий: 
Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.

**Задание 4.2.** 
Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [80]:
query = '''
    select count(v.id) as cnt
    from vacancies v 
    where salary_from is not null or salary_to is not null
    '''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1182742561.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,cnt
0,24073


У 24073 вакансий заполнено хотя бы одно из двух полей с зарплатой. Это примерно половина всех вакансий.

**Задание 4.3.** Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого числа.

In [81]:
query = '''
    select round(avg(salary_from)) as salary_from, round(avg(salary_to)) as salary_to
    from vacancies v 
    '''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1409577755.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,salary_from,salary_to
0,71065.0,110537.0


Среднее значение зарплатной вилки: от 71065р. до 110537р.

**Задание 4.4.**

Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. 
Какая пара находится на втором месте по популярности?

In [84]:
query = '''select count(id) as vacancies_count, schedule, employment
            from vacancies
            group by schedule, employment
            order by vacancies_count desc
            offset 1
            limit 1
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2733569308.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count,schedule,employment
0,7802,Удаленная работа,Полная занятость


На втором месте по популярности для сочетания типа рабочего графика и типа трудоустройства, используемого в вакансиях, находится пара "Удаленная работа	- Полная занятость", с количеством вакансий 7802.

**Задание 4.5.** Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [85]:
query = '''select experience, count(id) as vacancies_count
            from vacancies
            group by experience
            order by vacancies_count 
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\77004358.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Наибольшее количество вакансий представлено для опыта работы от 1 года до 3 лет: 26152 шт. 

Наименьшее количество вакансий - для специалистов с опытом работы более 6 лет: 1337 шт. 

*В HeadHunter представлено большое количество вакансий по всем регионам. Наибольшее количество вакансий в городах-милиоонниках.*

*Положительным моментом является то, что примерно в половине вакансий указан размер заработной платы. Это облегчит сооскателям поиск работы.*

*Возможен поиск работы по типу рабочего графика и типу трудоустройства.*

*Удобно искать вакансии с учетом требуемого опыта работы. Представлены вакансии без опыта работы, с опытом работы от 1 до 3 лет, от 3 до 6 лет и с опытом работы более 6 лет. Более половины вакансий в базе - с требуемым опытом работы от 1 года до 3 лет.*

## 5. Анализ работодателей

**Задание 5.1.** Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [86]:
query = '''select e.name as employer, count(v.id) as vacancies_count
            from vacancies v join employers e on v.employer_id = e.id
            group by employer
            order by vacancies_count desc
            limit 5
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\3544420121.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employer,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


На первом месте по количеству вакансий, с большим отрывом: Яндекс, 1933 вакансии. На пятом месте: Газпром нефть, 331 вакансия.

**Задание 5.2.** Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. 
Впишите его название в поле ниже в том виде, который вернул запрос.

In [89]:
query = '''select a.name as area, 
            count(e.id) as employers_count
            from employers e
                inner join areas a on e.area = a.id
                left join vacancies v on v.area_id = a.id
            group by a.name
            having count(v.id) = 0 
            order by employers_count desc
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1448924421.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,area,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
...,...,...
181,Удобная,1
182,Майна (Ульяновская область),1
183,Верхнетуломский,1
184,Жаркент,1


Среди регионов, в которых нет вакансий, по количеству работодателей лидирует регион "Россия": 410 работодателей.

**Задание 5.3.** Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.
Выберите максимальное значение из получившегося списка.

In [90]:
query = '''select e.name, count(distinct v.area_id) as areas_count
            from employers e 
            join areas a on e.area = a.id
            join vacancies v on e.id = v.employer_id
            group by e.name, a.name
            order by areas_count desc, e.name
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2382638078.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14897,Яроцкий Е. В./ MoonX,1
14898,Ясли-сад №374 г. Минска,1
14899,ЯСП,1
14900,Ястро-Переработка,1


Работодатель Яндекс лидирует по количеству регионов, в которых он публикует свои вакансии: 181 регион.

**Задание 5.4.** Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [92]:
query = '''select count(e.name) as employers_count
            from employers e
            left join employers_industries i on e.id = i.employer_id
            where i.industry_id is null
'''
df = pd.read_sql_query(query, connection)
display(df)


C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\3076194877.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,employers_count
0,8419


В базе существует большое количество работодателей, у которых не указана сфера деятельности: 8419 работодателей.

**Задание 5.5.** Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [93]:
query = '''select e.name  
            from employers e
            left join employers_industries i on e.id = i.employer_id
            group by e.name
            having count(i.industry_id) = 4
            order by 1
            offset 2
            limit 1
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\378952577.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name
0,2ГИС


Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности: 2ГИС.

**Задание 5.6.** С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [94]:
query = '''select count(e.id)
            from employers e
            join employers_industries ei on e.id = ei.employer_id
            join industries i on i.id = ei.industry_id
            where i.name = 'Разработка программного обеспечения'
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1434096468.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,3553


У 3553 работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

**Задание 5.7.**
Для компании «Яндекс» выведите список регионов-миллионников , в которых представлены вакансии компании, 
вместе с количеством вакансий в этих регионах. 
Также добавьте строку Total с общим количеством вакансий компании, собранных в этой таблице. 

In [96]:
query = '''select a.name, count(v.id) as cnt
            from employers e
            join vacancies v on v.employer_id = e.id
            join areas a on v.area_id = a.id
            where e.name = 'Яндекс' 
                and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 
                'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону',
                'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
            group by a.name
            union
            select 'Total', count(v.id)
            from employers e
            join vacancies v on v.employer_id = e.id
            join areas a on v.area_id = a.id
            where e.name = 'Яндекс' 
                and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 
                'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону',
                'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')
            order by 2, 1
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\1927579.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Казань,25
5,Пермь,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


Компания "Яндекс" предоставляет вакансии во всех городах-милионниках. Лидирует по количеству вакансий Москва. 
Всего компания "Яндекс" предлагает 485 вакансий в городах-милионниках.

*Компания Яндекс является абсолютным лидером по количеству вакансий. Также компания на первом месте по охвату регионов.*

*Существуют регионы, в которых нет вакансий, несмотря на большое количество работодателей в них. Для выяснения причин этого требуются дополнительные исследования.*

*В БД существует 8419 работодателей, у которых не указана сфера деятельности. Следует обратить на это внимание и исправить ситуацию. Исправление будет способствовать улучшению поиска вакансий.*

*Существует 3553 работодателя, у которых в качестве сферы деятельности указана "Разработка ПО".*

## 6. Предметный анализ

Переходим к самому интересному — нам предстоит исследовать требования работодателей к дата-сайентистам.

**Задание 6.1**
Сколько вакансий имеет отношение к данным?

In [97]:
query = '''select count(id) as vacancies_count
           from vacancies  
           where lower(name) like '%data%' or lower(name) like '%данн%' 
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\3708580572.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count
0,1771


1771 вакансия в БД имеет отношение к данным.

**Задание 6.2.** Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

* ‘data scientist’;
* ‘data science’;
* ‘исследователь данных’;
* ‘ML’ (здесь не нужно брать вакансии по HTML);
* ‘machine learning’;
* ‘машинн%обучен%’.

В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Считаем вакансиями для специалистов уровня Junior следующие:

* в названии есть слово “junior” или
* требуемый опыт — «Нет опыта» или
* тип трудоустройства — «Стажировка».

В качестве ответа запишите количество вакансий, которое вернул запрос.

In [98]:
query = '''select count(id) as vacancies_count
           from vacancies  
           where (lower(name) like '%data scientist%' 
            or lower(name) like '%data science%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and (lower(name) like '%junior%'
                or lower(experience) like '%нет опыта%' 
                or lower(employment) like '%стажировка%'
                )
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2058197665.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count
0,51


В базе представлена 51 вакансия для дата-сайентистов для специалистов уровня Junior.

**Задание 6.3**
Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [99]:
query = '''select count(id) as vacancies_count
           from vacancies  
           where (lower(name) like '%data scientist%' 
            or lower(name) like '%data science%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and (upper(key_skills) like '%SQL%' or lower(key_skills) like '%postgres%')
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\3015489079.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count
0,201


В базе представлена 201 вакансия для дата-сайентистов, где в качестве ключевого навыка указан SQL или postgres.

**Задание 6.4.**

С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. 
Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

In [100]:
query = '''select count(id) as vacancies_count
           from vacancies  
           where (lower(name) like '%data scientist%' 
            or lower(name) like '%data science%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and lower(key_skills) like '%python%'
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\767381372.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,vacancies_count
0,351


Количество вакансий, в которых в качестве ключевого навыка указан Python - 351 шт.

**Задание 6.5.**
Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [101]:
query = '''select round(avg(array_length(regexp_split_to_array(key_skills, '\t'), 1)), 2) as mean_skills_count
           from vacancies  
           where (lower(name) like '%data scientist%' 
            or lower(name) like '%data science%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2665476701.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,mean_skills_count
0,6.41


В вакансиях для DS работодатели в среднем указывают 6.41 ключевых навыков.

**Задание 6.6.** 

Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).
При решении задачи примите во внимание следующее:

1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null).
Чтобы избежать этой ситуации, мы воспользуемся функцией coalesce , которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос select 1 + coalesce(null, 0).

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [103]:
query = '''
    select experience, 
        round(avg((coalesce(salary_from, 0) + coalesce(salary_to, 0)) / case when salary_from > 0 AND salary_to > 0 THEN 2 ELSE 1 END)) as mean_salary 
    from vacancies  
    where (lower(name) like '%data scientist%' 
            or lower(name) like '%data science%' 
            or lower(name) like '%исследователь данных%'
            or (name like '%ML%' and name not like '%HTML%')
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    group by experience
'''
df = pd.read_sql_query(query, connection)
display(df)

C:\Users\msa_i\AppData\Local\Temp\ipykernel_16360\2912488420.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience,mean_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


В представленных вакансиях средняя зарплата для DS стремительно растет в соответствии с требуемым опытом работы: 
от 74643р. без опыта работы до 243115р. с опытом работы от 3 до 6 лет.

*В HeadHunter представлена 1771 вакансия, имеющая отношение к данным. Из них только 51 вакансия подойдет для начинающих дата-сайентистов.*
*В 201 DS-вакансии в качестве ключевого навыка указан SQL и Postgres. В 351 вакансии Python указан в качестве ключевого навыка DS.*
*Среди вакансий по DS в среднем от соискателя требуется 6.41 ключевых навыков по специальности.*
*Согласно представленным по DS вакансиям, средняя зарплата DS-специалиста стремительно растет с опытом работы.*